# Minimum Eigen Optimizer


## Introduction

An interesting class of optimization problems to be addressed by quantum computing are Quadratic Unconstrained Binary Optimization (QUBO) problems.
Finding the solution to a QUBO is equivalent to finding the ground state of a corresponding Ising Hamiltonian, which is an important problem not only in optimization, but also in quantum chemistry and physics. For this translation, the binary variables taking values in $\{0, 1\}$ are replaced by spin variables taking values in $\{-1, +1\}$, which allows to replace the resulting spin variables by Pauli Z matrices, and thus, an Ising Hamiltonian. For more details on this mapping we refere to [1].

Qiskit provides automatic conversion from a suitable `QuadraticProgram` to an Ising Hamiltonian, which then allows to leverage all the `MinimumEigenSolver` such as
- `VQE`,
- `QAOA`, or
- `NumpyMinimumEigensolver` (classical exact method).

Qiskit wraps the translation to an Ising Hamiltonian (in Qiskit Aqua also called `Operator`), the call to an `MinimumEigensolver` as well as the translation of the results back to `OptimizationResult` in the `MinimumEigenOptimizer`.

In the following we first illustrate the conversion from a `QuadraticProgram` to an `Operator` and then show how to use the `MinimumEigenOptimizer` with different `MinimumEigensolver` to solve a given `QuadraticProgram`.
The algorithms in Qiskit automatically try to convert a given problem to the supported problem class if possible, for instance, the `MinimumEigenOptimizer` will automatically translate integer variables to binary variables or add a linear equality constraints as a quadratic penalty term to the objective. It should be mentioned that Aqua will through a `QiskitOptimizationError` if conversion of a quadratic program with integer variable is attempted.

The circuit depth of `QAOA` potentially has to be increased with the problem size, which might be prohibitive for near-term quantum devices.
A possible workaround is Recursive QAOA, as introduced in [2].
Qiskit generalizes this concept to the `RecursiveMinimumEigenOptimizer`, which is introduced at the end of this tutorial.

### References
[1] [A. Lucas, *Ising formulations of many NP problems,* Front. Phys., 12 (2014).](https://arxiv.org/abs/1302.5843)

[2] [S. Bravyi, A. Kliesch, R. Koenig, E. Tang, *Obstacles to State Preparation and Variational Optimization from Symmetry Protection,* arXiv  preprint arXiv:1910.08980 (2019).](https://arxiv.org/abs/1910.08980)

## Converting a QUBO to an Operator

In [1]:
# Do the following imports:
# * Get BasicAer from Qiskit (for simulation)
# * Get QAOA, NumPyMinimumEigensolver from the Aqua's algorithms module
# * Get MinimumEigenOptimizer from the algorithms sub-module of Qiskit Optimization
# * Get the QuadraticProgram object from Qiskit Optimization
from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

In [2]:
# Declare an empty QuadraticProgram
qubo = QuadraticProgram()

# Add 3 binary variables: x,y, and z
qubo.binary_var('x')
qubo.binary_var('y')
qubo.binary_var('z')

# Using a mixture of arrays and dictionaries, declare the following
# objective function:
# x -2y + 3z + xy - xz +2yz ,
# which we will minimize
qubo.minimize(linear=[1,-2,3], quadratic={('x', 'y'): 1, ('x', 'z'): -1, ('y', 'z'): 2})

In [3]:
# Let's print the model!
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
0 <= x <= 1
0 <= y <= 1
0 <= z <= 1

Binaries
 x y z
End



Next we translate this QUBO into an Ising operator. This results not only in an `Operator` but also in a constant offset to be taking into account to shift the resulting value.

In [4]:
# Map the QUBO onto an Ising Hamiltonian
op, offset = qubo.to_ising()

In [5]:
# Let's look at these two things:
print('offset: {}'.format(offset))
print('operator:')
print(op)

offset: 1.5
operator:
SummedOp([
  -0.5 * IIZ,
  0.25 * IZI,
  -1.75 * ZII,
  0.25 * IZZ,
  -0.25 * ZIZ,
  0.5 * ZZI
])


Sometimes an `QuadraticProgram` might also directly be given in the form of an `Operator`. For such cases, Qiskit also provides a converter from an `Operator` back to a `QuadraticProgram`, which we illustrate in the following.

In [6]:
# Make an empty QuadraticProgram object
qp=QuadraticProgram()

# Injest the Ising Hamiltonian using the from_ising() method
qp.from_ising(op, offset, linear=True)

In [8]:
# The model is the same, up to different variables
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x_0 - 2 x_1 + 3 x_2 + [ 2 x_0*x_1 - 2 x_0*x_2 + 4 x_1*x_2 ]/2
Subject To

Bounds
0 <= x_0 <= 1
0 <= x_1 <= 1
0 <= x_2 <= 1

Binaries
 x_0 x_1 x_2
End



This converter allows, for instance, to translate an `Operator` to a `QuadraticProgram` and then solve the problem with other algorithms that are not based on the Ising Hamiltonian representation, such as the `GroverOptimizer`.

## Solving a QUBO with the MinimumEigenOptimizer

We start by initializing the `MinimumEigensolver` we want to use.

Here, we'll use two: `QAOA` (the quantum solver) and the `NumPyMinimumEigensolver` (for comparison to the exact solution).

In [9]:
# Declare the QAOA instance, using the statevector simulator of BasicAer
qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator'))

# Declare the exact instance by initializing the NumPyMinimumEigensolver
exact_mes = NumPyMinimumEigensolver()

Then, we use the `MinimumEigensolver` to create `MinimumEigenOptimizer`.

In [10]:
# Now, we set up the solver by plugging the instances into the MinEigenOptimizer object

# Using QAOA
qaoa = MinimumEigenOptimizer(qaoa_mes) 

# Using the exact classical solver
exact = MinimumEigenOptimizer(exact_mes)

Let's solve the problem exactly, which gives us a solution we can compare the QAOA-based solver to.

In [12]:
# Solve it!
exact_result = exact.solve(qubo)

In [15]:
# Let's look at the result
print(exact_result)

optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


Next we apply the `MinimumEigenOptimizer` based on `QAOA` to the same problem.

In [14]:
# Notice how we can use the same .solve() method?
qaoa_result = qaoa.solve(qubo)

In [16]:
# Let's look at the result.
print(qaoa_result)

optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


The QAOA-based solver matches the exact one.

In [17]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright